# Environment
Load packages.

In [ ]:
# Initialize workers
num_of_procs = 8 # You can also use addprocs() with no argument to create as many workers as your threads
using Distributed
addprocs(num_of_procs, exeflags="--project") # initial workers with the project env in current work directory

using ParallelExperiment

@everywhere[
    using POMDPs
    using POMDPSimulators
    using ParticleFilters
    using POMDPPolicies
    using BeliefUpdaters 
    
    using POMCPOW
    using BasicPOMCP
    using PL_DESPOT
    using AdaOPS
    using QMDP
    ]

# POMDP related pkgs
# For roomba and BabyPOMDP belief updater

# For visualization
using D3Trees
using POMDPModelTools
using POMDPGifs
import Cairo,Fontconfig

# For data processing and storing
using Statistics
using DataFrames
using CSV
using Random
using Printf

┌ Info: Precompiling PL_DESPOT [e87855f3-d877-40b7-8600-5f1cc2d263ad]
└ @ Base loading.jl:1278
┌ Info: Precompiling AdaOPS [eadfb9d8-44f1-454c-a5eb-0663ee7d74a1]
└ @ Base loading.jl:1278


# VDPTag2 Env

In [ ]:
# UCT-DESPOT
@everywhere push!(LOAD_PATH, "../../VDPTag2.jl")
@everywhere using VDPTag2
using Plots
using Reel
using ProgressMeter

# VDPTag2 Setting

In [ ]:
pomdp = VDPTagPOMDP()

# For POMCPOW
random_value_estimator = FORollout(RandomPolicy(pomdp))
value_estimator = FORollout(ToNextML(pomdp))
pomcpow_dict = Dict(:default_action=>[RandomPolicy(pomdp),],
                    :estimate_value=>[random_value_estimator],
                    :tree_queries=>[200000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(30.),])

# Solver list
solver_list = [POMCPOWSolver=>pomcpow_dict]

# VDPTag2 Visualization

In [ ]:
# solver = POMCPOWSolver(tree_queries=200000, max_time=1.0, criterion=MaxUCB(30), estimate_value=random_value_estimator)

# planner = solve(solver, pomdp)
# hr = HistoryRecorder(max_steps=30)
# belief_updater = SIRParticleFilter(pomdp, 2000)
# hist = POMDPs.simulate(hr, pomdp, planner, belief_updater)

# frames = Frames(MIME("image/png"), fps=2)
# gr()
# @showprogress "Creating gif..." for i in 1:n_steps(hist)
#     push!(frames, plot(pomdp, view(hist, 1:i)))
# end

# filename = string("VDPTag2.gif")
# write(filename, frames)


In [ ]:
solver = POMCPOWSolver(tree_queries=200000,
                        max_time=1.0,
                        criterion=MaxUCB(30),
                        k_action=1,
                        alpha_action=0.2,
                        k_observation=1,
                        alpha_observation=0.2,
                        estimate_value=value_estimator,
                        tree_in_info=true)

planner = solve(solver, pomdp)
b0 = initialstate_distribution(pomdp)
a, info = action_info(planner, b0)
D3Tree(info[:tree], init_expand=5)

# Tests based on VDPTag2

In [ ]:
number_of_episodes = 100
max_steps = 100
rng = MersenneTwister(1)

dfs = parallel_experiment(pomdp,
                          number_of_episodes,
                          max_steps,
                          solver_list,
                          initialstate=initialstate(pomdp, rng),
                          full_factorial_design=false)

CSV.write("VDPTag2_POMCPOW.csv", dfs[1])

# Discrete VDPTag2 Setting

In [ ]:
pomdp = AODiscreteVDPTagPOMDP()

# To-do
# Transplant ManageUncertainty policy

# For LB-DESPOT
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 100.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[RandomPolicy(pomdp),], 
                    :bounds=>[random_bounds],
                    :K=>[500, 300],
                    :beta=>[0.5, 0., 0.1, 1., 5.])

# For UCT-DESPOT
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:default_action=>[RandomPolicy(pomdp),],
                    :rollout_policy=>[random_rollout_policy],
                    :max_trials=>[100000,],
                    :K=>[500, 1000],
                    :m=>[10, 30],
                    :c=>[10.,])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict]

# Discrete VDPTag2 Visualization

In [ ]:
# solver = LB_DESPOTSolver(bounds=random_bounds, beta=0.5, K=100, default_action=RandomPolicy(pomdp))
# solver = UCT_DESPOTSolver(rollout_policy=random_rollout_policy, max_trials=100000, m=10, K=500, c=10)

# planner = solve(solver, pomdp)
# hr = HistoryRecorder(max_steps=30)
# belief_updater = SIRParticleFilter(pomdp, 2000)
# hist = POMDPs.simulate(hr, pomdp, planner, belief_updater)

# frames = Frames(MIME("image/png"), fps=2)
# gr()
# @showprogress "Creating gif..." for i in 1:n_steps(hist)
#     push!(frames, plot(pomdp, view(hist, 1:i)))
# end

# filename = string("Discrete_VDPTag2.gif")
# write(filename, frames)

In [ ]:
# solver = LB_DESPOTSolver(bounds=random_bounds, beta=0.0, K=100, default_action=RandomPolicy(pomdp), tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=random_rollout_policy, max_trials=100000, m=10, K=500, c=10, tree_in_info=true)

# planner = solve(solver, pomdp)
# b0 = initialstate(pomdp)
# a, info = action_info(planner, b0)
# println("number of trials: $(info[:record][1])")
# println("time for building DESPOT: $(info[:record][2])")
# D3Tree(info[:tree], init_expand=5)

# Tests based on Discrete VDPTag2

In [ ]:
number_of_episodes = 100
max_steps = 100
rng = MersenneTwister(1)

dfs = parallel_experiment(pomdp,
                          number_of_episodes,
                          max_steps,
                          solver_list,
                          initialstate=initialstate(pomdp, rng),
                          full_factorial_design=false)

CSV.write("DiscreteVDPTag2_LB-DESPOT.csv", dfs[1])
CSV.write("DiscreteVDPTag2_UCT-DESPOT.csv", dfs[2])

# RockSample Setting

In [ ]:
# @everywhere push!(LOAD_PATH, "../../RockSample.jl")
@everywhere using RockSample
pomdp = RockSamplePOMDP(map_size=(7,8),
                        rocks_positions=[(2,3), (1,8), (4,5), (5,2), (7,7)], 
                        sensor_efficiency=20.0,
                        discount_factor=0.95, 
                        good_rock_reward = 20.0)

# pomdp = RockSamplePOMDP(map_size=(11,11),
#                         rocks_positions=[(2,8), (1,6), (4,9), (5,2), (8,7), (9,10), (11,2)], 
#                         sensor_efficiency=20.0,
#                         discount_factor=0.95, 
#                         good_rock_reward = 20.0)

# pomdp = RockSamplePOMDP(map_size=(15,15),
#                         rocks_positions=[(2,3), (1,12), (4,1), (5,5), (8,15), (9,14), (12,1), (14,15), (15,7)], 
#                         sensor_efficiency=20.0,
#                         discount_factor=0.95, 
#                         good_rock_reward = 20.0)


# QMDP upper bound
qmdp_policy = solve(QMDPSolver(), pomdp)
function qmdp_upper_bound(pomdp, b)
    return value(qmdp_policy, b)
end

# default policy
move_east = FunctionPolicy() do b
    return 2
end

to_best = FunctionPolicy() do b 
    if typeof(b) <: RSState 
        s = b 
        val, ind = findmax(s.rocks) 
    else 
        s = rand(b) 
        good_count = zeros(Int, length(s.rocks)) 
        for state in particles(b) 
            good_count += state.rocks 
        end 
        val, ind = findmax(good_count) 
    end 
    if val/length(s.rocks) < 0.5 
        return 2 
    end 
    rock_pos = pomdp.rocks_positions[ind]
    diff = rock_pos - s.pos 
    if diff[2] != 0 
        if sign(diff[2]) == 1 
            return 1 # to north 
        else 
            return 3 # to south 
        end 
    else 
        if sign(diff[1]) == 1 
            return 2 # to east 
        elseif sign(diff[1]) == -1 
            return 4 # to west 
        else 
            return 5 # sample 
        end 
    end 
end

# For LB-DESPOT
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 30.0, check_terminal=true)
bounds = IndependentBounds(DefaultPolicyLB(to_best), 30.0, check_terminal=true)
bounds_hub = IndependentBounds(DefaultPolicyLB(to_best), qmdp_upper_bound, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[RandomPolicy(pomdp),], 
                    :bounds=>[random_bounds],
                    :K=>[300, 100],
                    :beta=>[0.5, 0., 0.1, 1., 5.])

# For UCT-DESPOT
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict1 = Dict(:default_action=>[RandomPolicy(pomdp),],
                    :rollout_policy=>[random_rollout_policy],
                    :max_trials=>[100000,],
                    :K=>[1000, 2000],
                    :m=>[50, 100],
                    :c=>[1, 10.])
uctdespot_dict2 = Dict(:default_action=>[RandomPolicy(pomdp),],
                    :rollout_policy=>[random_rollout_policy],
                    :max_trials=>[100000,],
                    :K=>[300, 100, 500],
                    :m=>[50, 30],
                    :c=>[1.,10,])
# For POMCPOW
random_value_estimator = FORollout(RandomPolicy(pomdp))
value_estimator = FORollout(to_best)
pomcpow_dict = Dict(:default_action=>[RandomPolicy(pomdp),],
                    :estimate_value=>[random_value_estimator],
                    :tree_queries=>[200000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(10.),])

# Solver list
solver_list = [#LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict1,
                UCT_DESPOTSolver=>uctdespot_dict2,] 
                #POMCPOWSolver=>pomcpow_dict]

# RockSample Visualization

In [ ]:
# solver = LB_DESPOTSolver(bounds=random_bounds, beta=0.5, K=100, default_action=RandomPolicy(pomdp))
# solver = UCT_DESPOTSolver(rollout_policy=random_rollout_policy, max_trials=100000, m=10, K=500, c=10)
# solver = POMCPOWSolver(tree_queries=200000, max_time=1.0, criterion=MaxUCB(30), estimate_value=random_value_estimator)

# planner = solve(solver, pomdp)
# makegif(pomdp, planner, filename="rock_sample.gif", max_steps=100, show_progress=true)

In [ ]:
solver = LB_DESPOTSolver(bounds=bounds_hub, beta=0.5, K=100, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=random_rollout_policy, max_trials=100000, m=50, K=1000, c=1, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=200000, max_time=1.0, criterion=MaxUCB(10), estimate_value=value_estimator, tree_in_info=true)

planner = solve(solver, pomdp)
b0 = initialstate(pomdp)
a, info = action_info(planner, b0)
# println("number of trials: $(info[:record][1])")
# println("time for building DESPOT: $(info[:record][2])")
D3Tree(info[:tree], init_expand=5)

# Tests based on RockSample

In [ ]:
number_of_episodes = 100
max_steps = 100
rng = MersenneTwister(1)
uctdespot_dict = Dict(:default_action=>[RandomPolicy(pomdp),],
                    :rollout_policy=>[random_rollout_policy],
                    :max_trials=>[100000,],
                    :K=>[300,],
                    :m=>[20,],
                    :c=>[10.,])
solver_list = [UCT_DESPOTSolver=>uctdespot_dict,] 
dfs = parallel_experiment(pomdp,
                          number_of_episodes,
                          max_steps,
                          solver_list,
                          initialstate=initialstate(pomdp, rng),
                          full_factorial_design=true)

# CSV.write("RockSample_LB-DESPOT.csv", dfs[1])
CSV.write("RockSample_UCT-DESPOT.csv", dfs[1])
# CSV.write("RockSample_POMCPOW.csv", dfs[3])

# BabyPOMDP Setting
Setting up a BabyPOMDP problem for further using.

In [ ]:
@everywhere using POMDPModels # For BabyPOMDP
pomdp = BabyPOMDP(-5, -10, 0.1, 0.8, 0.1, 0.95) # defualt setting except that the discount is 0.95

# Feed When Crying Policy
@everywhere function feed_when_crying(b)
    if typeof(b) == Bool
        b 
    elseif (typeof(b) <: UCTDESPOT.ScenarioBelief || typeof(b) <: LBDESPOT.ScenarioBelief)
        if typeof(currentobs(b)) <: Bool
            currentobs(b)
        else
            pdf(currentobs(b), true) > 0.5
        end
    elseif typeof(b) <: POMDPModels.BoolDistribution
        rand(b)
    else
        pdf(b, true) > 0.5
    end
end
feed_when_crying_policy = solve(FunctionSolver(feed_when_crying), pomdp)

# For LB-DESPOT
# Assume all following rewards are coming from the worst case, "hungry but don't feed"
@everywhere fval(m::BabyPOMDP, x) = reward(m, true, false)/(1-discount(m))
bounds = IndependentBounds(DefaultPolicyLB(feed_when_crying_policy, final_value=fval), 0.0)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp), final_value=fval), 0.0)
lbdespot_dict = Dict(:default_action=>[feed_when_crying_policy,], 
                    :bounds=>[bounds, random_bounds],
                    :K=>[1000, 2000, 3600, 5000],
                    :beta=>[0., 0.1, 0.5, 1., 5.])

# For UCT-DESPOT
rollout_policy = feed_when_crying_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :K=>[1000, 2000, 3600, 5000],
                        :m=>[30, 60, 100],
                        :c=>[500., 1000., 2000.])

# For POMCPOW
value_estimator = PORollout(feed_when_crying_policy, PreviousObservationUpdater())
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[200000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(0.1), MaxUCB(1.0), MaxUCB(10.), MaxUCB(100.), MaxUCB(1000.)])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict,
                QMDPSolver=>Dict(:max_iterations=>[200,]),
                FuncSolver=>Dict(:func=>[feed_when_crying,])]

# BabyPOMDP Visualization
Visualize BabyPOMDP in form of a tree.

In [ ]:
solver = LB_DESPOTSolver(bounds=bounds, beta=0.0, K=3000, default_action=feed_when_crying_policy, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, m=1, K=1000, c=1000, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=200000, max_time=1.0, criterion=MaxUCB(1), estimate_value=value_estimator, tree_in_info=true)

planner = solve(solver, pomdp)
b0 = initialstate_distribution(pomdp)
a, info = action_info(planner, b0)
# println("number of trials: $(info[:record][1])")
# println("time for building DESPOT: $(info[:record][2])")
# println("tree depth: $(info[:record][3])")
D3Tree(info[:tree], init_expand=5)

# Tests based on BabyPOMDP
First, run a simulation of one episode and one max step in case there's some latent bugs.

In [ ]:
number_of_episodes = 1
max_steps = 1
rng = MersenneTwister(1)

dfs = parallel_experiment(pomdp,
                          number_of_episodes,
                          max_steps,
                          solver_list,
                          initialstate=initialstate(pomdp, rng),
                          full_factorial_design=false)

# CSV.write("LaserTag_LB-DESPOT.csv", dfs[1])
CSV.write("LaserTag_UCT-DESPOT.csv", dfs[2])
# CSV.write("LaserTag_POMCPOW.csv", dfs[3])
# CSV.write("LaserTag_QMDP.csv", dfs[4])
# CSV.write("LaserTag_Move_Towards.csv", dfs[5])

# LaserTag Setting

In [2]:
@everywhere using LaserTag
pomdp = gen_lasertag()
belief_updater = SIRParticleFilter(pomdp, 10000)

# Policies
@everywhere function move_towards(b)
    s = typeof(b) <: LTState ? b : rand(b)
    
    # try to sneak up diagonally
    diff = s.opponent-s.robot
    dx = diff[1]
    dy = diff[2]
    if abs(dx) == 1 && abs(dy) == 1
        LaserTag.DIR_TO_ACTION[[dx, dy]]
    elseif abs(dx) == 1
        LaserTag.DIR_TO_ACTION[[0, sign(dy)]]
    elseif abs(dy) == 1
        LaserTag.DIR_TO_ACTION[[sign(dx), 0]]
    else
        LaserTag.DIR_TO_ACTION[[sign(dx), sign(dy)]]
    end
end
move_towards_policy = solve(FunctionSolver(move_towards), pomdp)

# QMDP upper bound
qmdp_policy = solve(QMDPSolver(), pomdp)
function qmdp_upper_bound(pomdp, b)
    return value(qmdp_policy, b)
end


# For AdaOPS
bounds = AdaOPS.IndependentBounds(FORollout(move_towards_policy), 10.0, check_terminal=true)
bounds_hub = AdaOPS.IndependentBounds(FORollout(move_towards_policy), qmdp_upper_bound, check_terminal=true)
random_bounds = AdaOPS.IndependentBounds(FORollout(RandomPolicy(pomdp)), 10.0, check_terminal=true)
ops_dict = Dict(:default_action=>[move_towards_policy,],
                    :bounds=>[bounds, random_bounds],
                    :m=>[100, 300],
                    )


# For POMCPOW
value_estimator = POMCPOW.FORollout(move_towards_policy)
random_value_estimator = POMCPOW.FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[150000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(100),],
                    :enable_action_pw=>[false,],
                    :k_observation=>[2.,],
                    :alpha_observation=>[0.15,])

# Solver list
solver_list = [OPSSolver=>ops_dict, 
                POMCPOWSolver=>pomcpow_dict,
                QMDPSolver=>Dict(:max_iterations=>[200,]),
                FuncSolver=>Dict(:func=>[move_towards,])]

┌ Info: Precompiling LaserTag [40beb472-b4db-4f0b-8d48-5bd2f96ebe44]
└ @ Base loading.jl:1278
┌ Warning: Module ParticleFilters with build ID 84477318880881 is missing from the cache.
│ This may mean ParticleFilters [c8b314e2-9260-5cf8-ae76-3be7461ca6d0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Info: Skipping precompilation since __precompile__(false). Importing LaserTag [40beb472-b4db-4f0b-8d48-5bd2f96ebe44].
└ @ Base loading.jl:1034


      From worker 3:	┌ Warning: Module ParticleFilters with build ID 84479085154914 is missing from the cache.
      From worker 3:	│ This may mean ParticleFilters [c8b314e2-9260-5cf8-ae76-3be7461ca6d0] does not support precompilation but is imported by a module that does.
      From worker 3:	└ @ Base loading.jl:1017
      From worker 2:	┌ Warning: Module ParticleFilters with build ID 84478668315639 is missing from the cache.
      From worker 2:	│ This may mean ParticleFilters [c8b314e2-9260-5cf8-ae76-3be7461ca6d0] does not support precompilation but is imported by a module that does.
      From worker 2:	└ @ Base loading.jl:1017
      From worker 9:	┌ Warning: Module ParticleFilters with build ID 84479017058247 is missing from the cache.
      From worker 9:	│ This may mean ParticleFilters [c8b314e2-9260-5cf8-ae76-3be7461ca6d0] does not support precompilation but is imported by a module that does.
      From worker 9:	└ @ Base loading.jl:1017
      From worker 7:	┌ Warning: Module P

4-element Array{Pair{Any,Dict{Symbol,V} where V},1}:
              OPSSolver => Dict{Symbol,Array{T,1} where T}(:m => [100, 300],:default_action => [FunctionPolicy{typeof(move_towards)}(move_towards)],:bounds => OPS.IndependentBounds{FORollout,Float64}[OPS.IndependentBounds{FORollout,Float64}(FORollout(FunctionPolicy{typeof(move_towards)}(move_towards)), 10.0, true, 0.0, nothing), OPS.IndependentBounds{FORollout,Float64}(FORollout(RandomPolicy{Random._GLOBAL_RNG,LaserTagPOMDP{DESPOTEmu,StaticArrays.MArray{Tuple{8},Int64,1,8}},NothingUpdater}(Random._GLOBAL_RNG(), LaserTagPOMDP{DESPOTEmu,StaticArrays.MArray{Tuple{8},Int64,1,8}}
  tag_reward: Float64 10.0
  step_cost: Float64 1.0
  discount: Float64 0.95
  floor: LaserTag.Floor
  obstacles: Set{StaticArrays.SArray{Tuple{2},Int64,1,2}}
  robot_init: Nothing nothing
  diag_actions: Bool false
  dcache: LaserTag.LTDistanceCache
  obs_model: DESPOTEmu
, NothingUpdater())), 10.0, true, 0.0, nothing)])
          POMCPOWSolver => Dict{Symbol,Ar

# LaserTag Visualization

In [13]:
solver = OPSSolver(bounds=bounds, delta=1, m=1000, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=1000000,
#                         max_time=1.0,
#                         criterion=MaxUCB(100),
#                         estimate_value=value_estimator,
#                         enable_action_pw=false,
#                         k_observation=2,
#                         alpha_observation=0.15,
#                         tree_in_info=true)

planner = solve(solver, pomdp)
b0 = initialstate_distribution(pomdp)
@time a, info = action_info(planner, b0)
# println("number of trials: $(first(info[:record]))")
# println("time for building DESPOT: $(last(info[:record]))")
D3Tree(info[:tree], init_expand=5)

  3.701988 seconds (27.66 M allocations: 2.141 GiB, 4.79% gc time)


o:<root> ESS:1000.00
L: -6.90, U:  8.50
├──a:1 r: -1.00
│  L: -8.08, U:  8.50
│  ├──o:[3, 2, 2, 0, 0, 4, 4, 4] ESS:  3.70
│  │  L: -7.49, U: 10.00
│  ├──o:[0, 0, 4, 0, 0, 0, 0, 0] ESS: 40.52
│  │  L:-14.97, U: 10.00
│  ├──o:[-1, -1, -1, -1, -1, -1, -1, -1] ESS:  6.00
│  │  L: 10.00, U: 10.00
│  ├──o:[0, 0, 0, 6, 0, 0, 0, 0] ESS: 24.30
│  │  L:-18.87, U: 10.00
│  ├──o:[0, 0, 4, 0, 0, 3, 0, 0] ESS: 36.66
│  │  L:-10.81, U:  8.50
│  │  ├──a:1 r: -1.00
│  │  │  L:-18.23, U:  8.49 (35 children)
│  │  ├──a:2 r: -1.00
│  │  │  L:-18.72, U:  8.50 (27 children)
│  │  ├──a:3 r: -1.00
│  │  │  L:-10.81, U:  8.50 (15 children)
│  │  ├──a:4 r: -1.00
│  │  │  L:-14.74, U:  8.50 (26 children)
│  │  └──a:5 r:-10.00
│  │     L:-27.38, U: -0.50 (27 children)
│  └──o:[0, 0, 6, 0, 0, 0, 0, 0] ESS: 17.23
│     L:-15.93, U: 10.00
├──a:2 r: -1.00
│  L: -6.90, U:  8.50
│  ├──o:[0, 0, 0, 2, 9, 0, 0, 5] ESS:  9.00
│  │  L: -6.18, U: 10.00
│  ├──o:[-1, -1, -1, -1, -1, -1, -1, -1] ESS:  7.00
│  │  L: 10.00, U: 10.00
│  ├──o:[0, 0, 5, 1, 0, 0, 0, 0] ESS: 46.14
│  │  L:-20.72, U: 10.00
│  ├──o:[0, 0, 5, 0, 0, 0, 0, 0] ESS: 47.36
│  │  L:-22.43, U: 10.00
│  └──o:[0, 0, 6, 0, 0, 0, 0, 0] ESS: 19.14
│     L:-26.29, U: 10.00
├──a:3 r: -1.00
│  L:-35.89, U:  8.50
│  ├──o:[1, 1, 1, 6, 0, 0, 0, 0] ESS:  1.89
│  │  L:-37.32, U: 10.00
│  ├──o:[-1, -1, -1, -1, -1, -1, -1, -1] ESS:  9.00
│  │  L: 10.00, U: 10.00
│  └──o:[0, 7, 0, 0, 5, 0, 0, 0] ESS: 29.19
│     L:-16.90, U: 10.00
├──a:4 r: -1.00
│  L:-13.62, U:  8.50
│  ├──o:[3, 6, 0, 0, 0, 1, 0, 0] ESS: 29.39
│  │  L:-13.51, U: 10.00
│  ├──o:[-1, -1, -1, -1, -1, -1, -1, -1] ESS: 10.00
│  │  L: 10.00, U: 10.00
│  └──o:[0, 6, 0, 0, 4, 0, 0, 0] ESS: 23.35
│     L:-15.01, U: 10.00
└──a:5 r: -9.70
   L:-20.15, U: -0.20
   ├──o:[-1, -1, -1, -1, -1, -1, -1, -1] ESS: 15.00
   │  L:  0.00, U: 10.00
   ├──o:[2, 0, 0, 4, 2, 1, 2, 0] ESS:  9.74
   │  L:-11.15, U: 10.00
   └──o:[0, 0, 5, 0, 0, 0, 0, 0] ESS: 22.53
      L:-14.27, U: 10.00

# Tests based on LaserTag

In [ ]:
number_of_episodes = 100
max_steps = 100
rng = MersenneTwister(1)

dfs = parallel_experiment(pomdp,
                          number_of_episodes,
                          max_steps,
                          solver_list,
                          belief_updater=belief_updater,
                          initialstate=initialstate(pomdp, rng),
                          full_factorial_design=false)

# CSV.write("LaserTag_LB-DESPOT.csv", dfs[1])
CSV.write("LaserTag_UCT-DESPOT.csv", dfs[1])
CSV.write("LaserTag_POMCPOW.csv", dfs[2])
CSV.write("LaserTag_QMDP.csv", dfs[3])
CSV.write("LaserTag_Move_Towards.csv", dfs[4])

# Roomba Env

In [14]:
# Roomba related pkgs
# Roomba need ParticleFilters = "0.2" for compatibility
@everywhere using Roomba # For Roomba Env

LoadError: ArgumentError: Package Roomba not found in current path:
- Run `import Pkg; Pkg.add("Roomba")` to install the Roomba package.


# Bumper Roomba Setting
Setting up a Roomba problem with bumper sensor for further using.\
The parameters of Roomba are listed as follows.
```
maximum velocity of Roomba [m/s]
v_max::Float64  = 10.0  # m/s

maximum turn-rate of Roombda [rad/s]
om_max::Float64 = 1.0   # rad/s

simulation time-step [s]
dt::Float64     = 0.5   # s

penalty for wall-contact
contact_pen::Float64 = -1.0 

penalty per time-step
time_pen::Float64 = -0.1

reward for reaching goal
goal_reward::Float64 = 10

penalty for reaching stairs
stairs_penalty::Float64 = -10

specifies room configuration (location of stairs/goal) {1,2,3}
config::Int = 1

environment room struct
room::Room  = Room(sspace,configuration=config)

environment state-space (ContinuousRoombaStateSpace or DiscreteRoombaStateSpace)
sspace::SS = ContinuousRoombaStateSpace()

environment action-space struct
aspace::AS = RoombaActions()
```

In [ ]:
max_speed = 2.0
speed_interval = 2.0
max_turn_rate = 1.0
turn_rate_interval = 1.0

sensor = Bumper()
num_particles = 5000 # number of particles in belief

pos_noise_coeff = 0.3
ori_noise_coeff = 0.1

# POMDP problem
action_space = vec([RoombaAct(v, om) for v in 0:speed_interval:max_speed, om in -max_turn_rate:turn_rate_interval:max_turn_rate])
pomdp = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(aspace=action_space));

# Belief updater
resampler = BumperResampler(num_particles, pomdp, pos_noise_coeff, ori_noise_coeff)
belief_updater = BasicParticleFilter(pomdp, resampler, num_particles)

# Rush Policy
rush_policy = FunctionPolicy() do b
    if !(typeof(b) <: ParticleFilters.ParticleCollection) &&
        !(typeof(b) <: Roomba.RoombaInitialDistribution) &&
        b !== nothing &&
        typeof(b) == Bool ? b : (typeof(currentobs(b)) == Bool ? currentobs(b) : false)

        [max_speed, max_turn_rate]
    else
        [max_speed, 0.0]
    end
end

# For LB-DESPOT
bounds = IndependentBounds(DefaultPolicyLB(rush_policy), 10.0, check_terminal=true)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 10.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[rush_policy,], 
                    :bounds=>[bounds, random_bounds],
                    :K=>[100, 300, 500],
                    :beta=>[0., 0.1, 1., 10., 100.])

# For UCT-DESPOT
rollout_policy = rush_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :K=>[100, 300, 500],
                        :m=>[5, 10, 20, 30],
                        :c=>[0.1, 1., 10., 100., 1000., 10000.])

# For POMCPOW
value_estimator = PORollout(rush_policy, PreviousObservationUpdater())
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[100000,], 
                    :max_time=>[1.0,], 
                    :criterion=>[MaxUCB(0.1), MaxUCB(1.0), MaxUCB(10.), MaxUCB(100.), MaxUCB(1000.)])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict]

# Bumper Roomba Visualization

In [ ]:
# solver = LB_DESPOTSolver(bounds=bounds, default_action=rush_policy, tree_in_info=true)
solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=100000, max_time=1.0, estimate_value=value_estimator, tree_in_info=true)

planner = solve(solver, pomdp)
makegif(pomdp, planner, belief_updater, filename="bumper.gif", max_steps=100, show_progress=true)

In [ ]:
# solver = LB_DESPOTSolver(bounds=bounds, beta=0.2, T_max=60.0, K=100, default_action=rush_policy, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, T_max=60.0, m=10, K=100, c=100, tree_in_info=true)
# # solver = POMCPOWSolver(tree_queries=1000000, max_time=20.0, criterion=MaxUCB(1000), estimate_value=value_estimator, tree_in_info=true)

# planner = solve(solver, pomdp)
# b0 = initialstate_distribution(pomdp)
# @time a, info = action_info(planner, b0)
# @show info[:record]
# D3Tree(info[:tree], init_expand=5)

# Tests based on Bumper Roomba

In [ ]:
number_of_episodes = 1
max_steps = 1
rng = MersenneTwister(1)

dfs = parallel_experiment(pomdp,
                          number_of_episodes,
                          max_steps,
                          solver_list,
                          belief_updater=belief_updater,
                          initialstate=initialstate(pomdp, rng),
                          full_factorial_design=false)

for i in 1:length(dfs)
    CSV.write("BumperRoomba$(i).csv", dfs[i])
end

# Lidar Roomba Setting

In [ ]:
max_speed = 2.0
speed_interval = 2.0
max_turn_rate = 1.0
turn_rate_interval = 1.0

sensor = Lidar()
num_particles = 5000 # number of particles in belief

pos_noise_coeff = 0.3
ori_noise_coeff = 0.1

# POMDP problem
action_space = vec([RoombaAct(v, om) for v in 0:speed_interval:max_speed, om in -max_turn_rate:turn_rate_interval:max_turn_rate])
pomdp = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=2, aspace=action_space))

# Belief updater
resampler = LidarResampler(num_particles, pomdp, pos_noise_coeff, ori_noise_coeff)
belief_updater = BasicParticleFilter(pomdp, resampler, num_particles)

# Running policy
running_policy = FunctionPolicy() do b
    # s = typeof(b) == RoombaState ? b : typeof(b) <: AA228FinalProject.RoombaInitialDistribution ? rand(b) : mean(b)
    # The statement is computational inefficient.
    s = typeof(b) == RoombaState ? b : rand(b)
    # compute the difference between our current heading and one that would
    # point to the goal
    goal_x, goal_y = get_goal_xy(pomdp)
    x,y,th = s[1:3]
    ang_to_goal = atan(goal_y - y, goal_x - x)
    del_angle = wrap_to_pi(ang_to_goal - th)
    
    # apply proportional control to compute the turn-rate
    Kprop = 1.0
    om = Kprop * del_angle
    # find the closest option in action space
    _,ind = findmin(abs.(om .- (-max_turn_rate:turn_rate_interval:max_turn_rate)))
    om = (-max_turn_rate:turn_rate_interval:max_turn_rate)[ind]
    # always travel at some fixed velocity
    v = max_speed
    
    return RoombaAct(v, om)
end

# Roomba Upper Bound
function shortest_time(pomdp::RoombaPOMDP, b)
    s = typeof(b) == RoombaState ? b : rand(b)
    x,y,th = s[1:3]
    # point to the goal
    goal_x, goal_y = get_goal_xy(pomdp)
    shortest_dist = sqrt((goal_x - x)^2 + (goal_y - y)^2)
    return pomdp.goal_reward + pomdp.time_pen * shortest_dist / max_speed
end

# Roomba Initializer
function n_init(pomdp::RoombaPOMDP, h, a::RoombaAct)
    if a.v == 0 && a.omega == 0
        return 1
    end
    return 0
end
function v_init(pomdp::RoombaPOMDP, h, a::RoombaAct)
    if a.v == 0 && a.omega == 0
        return -100
    end
    return 0
end

function initializer(b, a::RoombaAct)
    return (v_init(pomdp, b, a), n_init(pomdp, b, a))
end

# For LB-DESPOT
bounds = IndependentBounds(DefaultPolicyLB(running_policy), 10.0, check_terminal=true)
# bounds with heuristic upper bound
bounds_hub = IndependentBounds(DefaultPolicyLB(running_policy), shortest_time, check_terminal=true)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 10.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[running_policy,], 
                    :bounds=>[bounds, random_bounds, bounds_hub],
                    :lambda=>[0.0, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                    :K=>[100, 300, 500],
                    :beta=>[0., 0.1, 1., 10., 100.])

# For UCT-DESPOT
rollout_policy = running_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :initializer=>[initializer,],
                        :K=>[100, 300, 500],
                        :m=>[5, 10, 20, 30],
                        :criterion=>[MaxUCB(0.1), MaxUCB(1.0), MaxUCB(10.), MaxUCB(100.), MaxUCB(1000.)])

# For POMCPOW
value_estimator = FORollout(running_policy)
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :init_N=>[n_init,],
                    :init_V=>[v_init,],
                    :tree_queries=>[100000,], 
                    :max_time=>[1.0,], 
                    :c=>[0.1, 1., 10., 100., 1000., 10000.])

# Solver list
solver_list = [LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, 
                POMCPOWSolver=>pomcpow_dict]

# Lidar Roomba Visualization

In [ ]:
# solver = LB_DESPOTSolver(bounds=bounds, default_action=running_policy)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy)
# solver = POMCPOWSolver(tree_queries=100000, max_time=1.0, estimate_value=value_estimator)

# planner = solve(solver, pomdp)
# makegif(pomdp, planner, belief_updater, filename="lidar.gif", max_steps=100, show_progress=true)

In [ ]:
# # solver = LB_DESPOTSolver(bounds=bounds, default_action=running_policy, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, initializer=initializer, tree_in_info=true)
# # solver = POMCPOWSolver(tree_queries=100000, max_time=1.0, estimate_value=value_estimator, tree_in_info=true)

# planner = solve(solver, pomdp)
# b0 = initialstate_distribution(pomdp)
# a, info = action_info(planner, b0)
# D3Tree(info[:tree], init_expand=5)

# Tests based on Lidar Roomba

In [ ]:
number_of_episodes = 1
max_steps = 1
rng = MersenneTwister(1)

dfs = parallel_experiment(pomdp,
                          number_of_episodes,
                          max_steps,
                          solver_list,
                          belief_updater=belief_updater,
                          initialstate=initialstate(pomdp, rng),
                          full_factorial_design=false)

for i in 1:length(dfs)
    CSV.write("LidarRoomba$(i).csv", dfs[i])
end

# Discrete Lidar Roomba

In [ ]:
max_speed = 2.0
speed_interval = 2.0
max_turn_rate = 1.0
turn_rate_interval = 1.0

cut_points =  exp10.(range(-.5, stop=1.3, length=10))
sensor = DiscreteLidar(cut_points)

num_particles = 5000 # number of particles in belief

pos_noise_coeff = 0.3
ori_noise_coeff = 0.1

# POMDP problem
action_space = vec([RoombaAct(v, om) for v in 0:speed_interval:max_speed, om in -max_turn_rate:turn_rate_interval:max_turn_rate])
pomdp = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=3, aspace=action_space));

# Belief updater
resampler = LidarResampler(num_particles, pomdp, pos_noise_coeff, ori_noise_coeff)
belief_updater = BasicParticleFilter(pomdp, resampler, num_particles)

# Running policy
running_policy = FunctionPolicy() do b
    # s = typeof(b) == RoombaState ? b : typeof(b) <: AA228FinalProject.RoombaInitialDistribution ? rand(b) : mean(b)
    # The statement is computational inefficient.
    s = typeof(b) == RoombaState ? b : rand(b)
    # compute the difference between our current heading and one that would
    # point to the goal
    goal_x, goal_y = get_goal_xy(pomdp)
    x,y,th = s[1:3]
    ang_to_goal = atan(goal_y - y, goal_x - x)
    del_angle = wrap_to_pi(ang_to_goal - th)
    
    # apply proportional control to compute the turn-rate
    Kprop = 1.0
    om = Kprop * del_angle
    # find the closest option in action space
    _,ind = findmin(abs.(om .- (-max_turn_rate:turn_rate_interval:max_turn_rate)))
    om = (-max_turn_rate:turn_rate_interval:max_turn_rate)[ind]
    # always travel at some fixed velocity
    v = max_speed
    
    return RoombaAct(v, om)
end

# Roomba Upper Bound
function shortest_time(pomdp::RoombaPOMDP, b)
    s = typeof(b) == RoombaState ? b : rand(b)
    x,y,th = s[1:3]
    # point to the goal
    goal_x, goal_y = get_goal_xy(pomdp)
    shortest_dist = sqrt((goal_x - x)^2 + (goal_y - y)^2)
    return pomdp.mdp.goal_reward + pomdp.mdp.time_pen * shortest_dist / max_speed
end

# Roomba Initializer
function n_init(pomdp::RoombaPOMDP, h, a::RoombaAct)
    if a.v == 0 && a.omega == 0
        return 1
    end
    return 0
end
function v_init(pomdp::RoombaPOMDP, h, a::RoombaAct)
    if a.v == 0 && a.omega == 0
        return -100.0
    end
    return 0.0
end

function initializer(b, a::RoombaAct)
    return (v_init(pomdp, b, a), n_init(pomdp, b, a))
end

# For LB-DESPOT
bounds = IndependentBounds(DefaultPolicyLB(running_policy), 10.0, check_terminal=true)
# bounds with heuristic upper bound
bounds_hub = IndependentBounds(DefaultPolicyLB(running_policy), shortest_time, check_terminal=true)
random_bounds = IndependentBounds(DefaultPolicyLB(RandomPolicy(pomdp)), 10.0, check_terminal=true)
lbdespot_dict = Dict(:default_action=>[running_policy,], 
                    :bounds=>[bounds, random_bounds],
                    :lambda=>[0.0, 0.01, 0.1, 1.0],
                    :T_max=>[30.0],
                    :K=>[100, 500],
                    :beta=>[0., 0.5, 1., 5.])

# For UCT-DESPOT
rollout_policy = running_policy
random_rollout_policy = RandomPolicy(pomdp)
uctdespot_dict = Dict(:rollout_policy=>[rollout_policy, random_rollout_policy],
                        :K=>[100, 500],
                        :T_max=>[30.0],
                        :m=>[10, 50],
                        :c=>[1., 10., 100.])

# For POMCPOW
value_estimator = FORollout(running_policy)
random_value_estimator = FORollout(RandomPolicy(pomdp))
pomcpow_dict = Dict(:estimate_value=>[value_estimator, random_value_estimator],
                    :tree_queries=>[100000,], 
                    :max_time=>[30.0,], 
                    :criterion=>[MaxUCB(0.1), MaxUCB(1.0), MaxUCB(10.), MaxUCB(100.), MaxUCB(1000.)])

# Solver list
solver_list = [#LB_DESPOTSolver=>lbdespot_dict, 
                UCT_DESPOTSolver=>uctdespot_dict, ]
                #POMCPOWSolver=>pomcpow_dict]

# Discrete Lidar Visualization

In [ ]:
solver = LB_DESPOTSolver(bounds=bounds_hub, beta=0.2, K=500, default_action=running_policy)
# solver = UCT_DESPOTSolver(m=10, K=100, c=100, T_max=1, rollout_policy=rollout_policy)
# solver = POMCPOWSolver(tree_queries=100000, max_time=1.0, estimate_value=value_estimator)

planner = solve(solver, pomdp)
makegif(pomdp, planner, belief_updater, filename="discrete_lidar.gif", max_steps=100, show_progress=true)

In [ ]:
solver = LB_DESPOTSolver(bounds=bounds_hub, T_max=10.0,lambda=1, beta=0.2, K=300, tree_in_info=true)
# solver = UCT_DESPOTSolver(rollout_policy=rollout_policy, initializer=initializer, T_max=10.0, m=30, K=1000, c=1, tree_in_info=true)
# solver = POMCPOWSolver(tree_queries=100000, max_time=10.0, init_N=n_init, init_V=v_init, criterion=MaxUCB(10), estimate_value=value_estimator, tree_in_info=true)

planner = solve(solver, pomdp)
b0 = initialstate_distribution(pomdp)
a, info = action_info(planner, b0)
# println("number of trials: $(first(info[:record]))")
# # @show info[:record]
# println("time for building DESPOT: $(last(info[:record]))")
D3Tree(info[:tree], init_expand=3)

# Tests based on Discrete Lidar Roomba

In [ ]:
number_of_episodes = 1
max_steps = 1
rng = MersenneTwister(1)

dfs = parallel_experiment(pomdp,
                          number_of_episodes,
                          max_steps,
                          solver_list,
                          belief_updater=belief_updater,
                          initialstate=initialstate(pomdp, rng),
                          full_factorial_design=false)

CSV.write("DiscreteLidarRoomba_LB_DESPOT.csv", dfs[1])
CSV.write("DiscreteLidarRoomba_UCT_DESPOT.csv", dfs[2])
CSV.write("DiscreteLidarRoomba_POMCPOW.csv", dfs[3])